In [33]:
import pandas as pd
from sqlalchemy import create_engine,VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text

#Origem
conexaoDB = ('DRIVER={ODBC Driver 17 for SQL Server};''SERVER=EDMILSON\MSSQLSERVER01;''DATABASE=AdventureWorksDW2019;''Trusted_Connection=yes;')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conexaoDB}')

query = '''
SELECT 
ProductKey
,ProductAlternateKey
,FrenchProductName
,EnglishProductName
,Color 
,StandardCost
  FROM [AdventureWorksDW2019].[dbo].[DimProduct]
  where StandardCost is not null
  and Color = 'Red'
  

'''
df = pd.read_sql_query(query, engine)

# Exiba  DataFrame
engine.dispose()
df.head()


,ProductKey,ProductAlternateKey,FrenchProductName,EnglishProductName,Color,StandardCost
0,212,HL-U509-R,"Casque sport 100, rouge","Sport-100 Helmet, Red",Red,12.0278
1,213,HL-U509-R,"Casque sport 100, rouge","Sport-100 Helmet, Red",Red,13.8782
2,214,HL-U509-R,"Casque sport 100, rouge","Sport-100 Helmet, Red",Red,13.0863
3,238,FR-R92R-62,"Cadre de vélo de route HL - rouge, 62","HL Road Frame - Red, 62",Red,747.9682
4,239,FR-R92R-62,"Cadre de vélo de route HL - rouge, 62","HL Road Frame - Red, 62",Red,722.2568


In [34]:
#conexao Destino - inferindo os tipos  dados 
conexao_destino = ('DRIVER={ODBC Driver 17 for SQL Server};''SERVER=EDMILSON\MSSQLSERVER01;''DATABASE=Python;''Trusted_Connection=yes;')
engine_destino = create_engine(f'mssql+pyodbc:///?odbc_connect={conexao_destino}')

#executar comando de delete
delete = text( """
delete  FROM [Python].[dbo].[produto_etl_historico] 
where color= 'Red'
"""
)
cursor = engine_destino.connect()
cursor.execute(delete)
cursor.commit()
cursor.close()

# Carga de dados 

tabela_destino= 'produto_etl_historico'
tipo_colnas = {        
    'ProductAlternateKey':VARCHAR(10),
    'Color':VARCHAR(15),
    'FrenchProductName':VARCHAR(50)
    }

df.to_sql(name= tabela_destino,con=engine_destino,if_exists='append' ,index=False , dtype=tipo_colnas)# enviar para banco 

engine_destino.dispose()
